# 클러스터에서 스파크 실행하기
- 이 장에서 알아볼 주제
  - 스파크 애플리케이션의 아키텍처와 컴포넌트
  - 스파크 내/외부에서 실행되는 스파크 애플리케이션의 생애주기
  - 파이프라이닝과 같은 중요한 저수준 실행 속성
  - 스파크 애플리케이션을 실행하는 데 필요한 사항

## 스파크 애플리케이션의 아키텍처
### 스파크 드라이버
  - 스파크 애플리케이션의 <strong>운전자 역할</strong>을 하는 프로세스
  - 스파크 애플리케이션의 <strong>실행을 제어</strong>하고
  - 스파크 클러스터(익스큐터의 상태와 태스크)의 <strong>모든 상태 정보를 유지</strong>하고
  - 물리적 컴퓨팅 자원 확보와 익스큐터 실행을 위해 <strong>클러스터 매니저와 통신</strong>할 수 있어야함

<br>

### 스파크 익스큐터
  - 스파크 익스큐터는 <strong>스파크 드라이버가 할당한 태스크를 수행</strong>하는 프로세스
  - 드라이버가 할당한 <strong>태스크를 받아 실행</strong>하고
  - 태스크의 <strong>상태와 결과(성공 또는 실패)를 드라이버에 보고</strong>함
  - 모든 스파크 애플리케이션은 <strong>개별 익스큐터 프로세스</strong>를 사용함

<br>

### 클러스터 매니저
  - 스파크 드라이버와 익스큐터를 허공에 띄울 수는 없으므로 클러스터 매니저가 필요
  - 클러스터 매니저는 <strong>스파크 애플리케이션을 실행할 클러스터 머신을 유지</strong>함
  - <strong>드라이버(마스터)와 워커</strong>라는 개념을 가지고 있음
  
  <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdTrTFM%2FbtqMgtI2j2i%2FEYx4YKL4F65mq2a0Yyx1LK%2Fimg.png" width=50%/>
  
    - 원은 개별 워커 노드를 실행하고 관리하는 데몬 프로세스
    - 위에 표시된 원들은 클러스터 매니저의 프로세스일 뿐
    
    
  - 스파크 애플리케이션을 실행할 때가 되면 클러스터 매니저에 자원 할당을 요청함
  - 스파크가 지원하는 클러스터 매니저
    - 스탠드얼론 클러스터 매니저
    - 아파치 메소스
    - 하둡 YARN

### 실행 모드
- 실행 모드는 애플리케이션을 실행할 때 요청한 자원의 물리적인 위치를 결정함
- 선택 가능한 실행 모드
  - 클러스터 모드
  - 클라이언트 모드
  - 로컬 모드

#### 클러스터 모드
- 가장 흔한 모드
- 클러스터 모드를 사용하려면 컴파일된 JAR파일이나 파이썬 스크립트 도는 R스크립트를 클러스터 매니저에 전달
- 클러스터 매니저는 파일은 받은 다음 <strong>워커 노드에 드라이버와 익스큐터 프로세스를 실행함</strong>
- 클러스터 매니저는 모든 스파크 애플리케이션과 관련된 <strong>프로세스를 유지</strong>하는 역할을 함



<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbJvmnR%2FbtqMmqdrL85%2FmlJL4vMXuDc0G3RGnWktwK%2Fimg.png" width=30% />
 
  - 위 그림은 하나의 워커 노드에 스파크 드라이버를 할당하고 다른 워커 노드에 익스큐터를 할당한 모습
  - 실선으로 그려진 직사각형: 드라이버 프로세스
  - 점선으로 그려진 직사각형: 익스큐터 프로세스

#### 클라이언트 모드
- 애플리케이션을 제출한 <strong>클라이언트 머신에 스파크 드라이버가 위치</strong>한다는 것을 제외하면 클러스터 모드와 비슷
  - 스파크 드라이버 프로세스를 유지하며 클러스터 매니저는 익스큐터 프로세스를 유지함
- 스파크 애플리케이션이 클러스터와 무관한 머신에서 동작
  - 보통 이런 머신을 게이트웨이 머신 또는 에지 노드라고 부름
  
  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb4iKEG%2FbtqMdDF3rDQ%2Fccqz2Y5UYwljpkKeaMDfXk%2Fimg.png" width=30% />
  
  
  - <strong>드라이버는 클러스터 외부의 머신에서 실행</strong>되고
  - 나머지 워커는 클러스터에 위치
  - 실선으로 그려진 직사각형: 드라이버 프로세스
  - 점선으로 그려진 직사각형: 익스큐터 프로세스

#### 로컬 모드
- 학습, 테스트 용도로 주로 사용
- 로컬 모드로 설정된 경우 모든 스파크 애플리케이션은 단일 머신에서 실행됨
- 로컬 모드는 애플리케이션의 병렬 처리를 위해 <strong>단일 머신의 스레드</strong>를 활용

## 스파크 애플리케이션의 생애주기(스파크 외부)
- 초기화부터 종료까지 스파크 애플리케이션의 생애주기를 단계별로 볼 것임

### 1단계: 클라이언트 요청

<img src ="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FIn7jj%2FbtqMdVNdQ8R%2FVfs1WRjog7iO7NIaYssKTk%2Fimg.png" width =30% />

<br>

  - 두꺼운 화살표 선: 스파크나 스파크 관련 프로세스가 수행하는 통신
  - 실선: 클러스터 매니저와의 통신 같은 일반적인 통신


#### 1. 스파크 애플리케이션(컴파일된 JAR나 라이브러리 파일) 제출
  - 제출하는 시점에 로컬 머신에서 코드가 실행되어 클러스터 드라이버 노드에 요청함
  - 이 과정에서 스파크 드라이버 프로세스의 자원을 함께 요청

#### 2. 클러스터 매니저는 이 요청을 받아들이고 클러스터 노드 중 하나에 드라이버 프로세스를 실행
#### 3. 스파크 잡을 제출한 클라이언트 프로세스는 종료되고 애플리케이션은 클러스터에서 실행

### 2단계: 시작

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbCCLbp%2FbtqMknad0vw%2Fn1tjDxHakavvfmZUdtmxy0%2Fimg.png" width=30% />

<br>

- 사용자 코드에는 <strong>반드시 스파크 클러스터(드라이버와 익스큐터)를 초기화하는 SparkSession이 포함</strong>되어야 함
  - SparkSession은 <strong>클러스터 매니저와 통신하여 스파크 익스큐터 프로세스의 실행을 요청</strong>함
  - 사용자는 spark-submit을 실행할 때 사용하는 명령행 인수로 익스큐터 수와 설정값을 지정 가능
  
#### 1. 클러스터 매니저는 익스큐터 프로세스를 시작하고 
#### 2. 결과를 응답 받아 익스큐터의 위치와 관련된 정보를 드라이버 프로세스로 전송
#### 3. 모든 작업이 정상적으로 완료되면 '스파크 클러스터'가 완성

### 3단계: 실행

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbSMFvs%2FbtqMgtWyyzw%2FiDqCWVEspmLMl0Unopfajk%2Fimg.png" width=30% />

<br>

#### 1. 드라이버와 워커는 코드를 실행하고 데이터를 이동하는 과정에서 서로 통신
#### 2. 드라이버는 각 워커에 태스크 할당
#### 3. 태스크를 할당받은 워커는 태스크의 상태와 성공/실패 여부를 드라이버에 전송

### 4단계: 완료

#### 1. 스파크 애플리케이션의 실행이 완료되면 드라이버 프로세스가 성공이나 실패 중 하나의 상태로 종료됨
#### 2. 그 다음 클러스터 매니저는 드라이버가 속한 스파크 클러스터의 모든 익스큐터를 종료시킴
#### 3. 이 시점에 스파크 애플리케이션의 성공/실패 여부를 클러스터 매니저에 요청해 확인 가능

## 스파크 애플리케이션의 생애주기(스파크 내부)
- 스파크 애플리케이션을 정의하는 실제 '사용자 코드'와 관련된 내용을 다룰 것임
- 스파크 애플리케이션은 하나 이상의 스파크 잡으로 구성됨
- 스레드를 사용해 여러 액션을 병렬로 수행하는 경우가 아니라면 애플리케이션의 스파크 잡은 차례대로 실행됨

### SparkSession
- 모든 스파크 애플리케이션은 가장 먼저 <strong>SparkSession</strong>을 생성함
  - 여러 대화형 모드에선 자동으로 생성
  - 애플리케이션을 만드는 경우라면 직접 생성

In [0]:
from pyspark.sql import SparkSession
#SparkSession 생성하기
spark = SparkSession.builder.master('local').appName('word count').config('spark.some.config.option','some-value').getOrCreate()

#### SparkContext
- SparkSession의 SparkContext는 스파크 클러스터에 대한 <strong>연결</strong>을 나타냄
- SparkContext를 이용해 RDD같은 스파크의 저수준 API를 사용할 수 있음
  - 또한 브로드캐스트 변수를 생성하고 코드를 실행할 수 있음
- getOrCreate메서드로 직접 SparkContext초기화 가능

#### 논리적 명령
- 스파크 코드는 트랜스포메이션과 액션으로 구성됨
- 사용자는 SQL, 저수준 RDD 처리, 머신러닝 알고리즘 등을 사용해 트랜스포메이션과 액션을 마음대로 구성할 수 있음
- DataFrame과 같은 <strong>선언적 명령을 사용하는 방법과 논리적 명령이 물리적 실행 계획으로 어떻게 변환되는지</strong> 이해하는 것은 중요

In [0]:
df1= spark.range(2,10000000, 2)
df2= spark.range(2,10000000, 4)

#세 단계의 잡
## 파티션을 재분배하는 잡
step1= df1.repartition(5)
step12 = df2.repartition(6)

## 값을 트랜스포메이션하는 잡
step2=step1.selectExpr('id*5 as id')
step3 = step2.join(step12,['id'])

## 집계 및 최종결과를 얻어내는 잡
step4 = step3.selectExpr('sum(id)')
## 액션
step4.collect()

Out[ 2 ]: [Row(sum(id)=2500000000000)]

----
- 액션을 호출하면 개별 스테이지와 태스크로 이루어진 스파크 잡이 실행됨

In [0]:
step1.explain()

== Physical Plan ==
Exchange RoundRobinPartitioning(5)
+- *(1) Range (2, 10000000, step=2, splits=16)

In [0]:
step12.explain()

== Physical Plan ==
Exchange RoundRobinPartitioning(6)
+- *(1) Range (2, 10000000, step=4, splits=16)

In [0]:
step2.explain()

== Physical Plan ==
*(2) Project [(id#204L * 5) AS id#210L]
+- Exchange RoundRobinPartitioning(5)
 +- *(1) Range (2, 10000000, step=2, splits=16)

In [0]:
step4.explain()

== Physical Plan ==
*(5) HashAggregate(keys=[], functions=[finalmerge_sum(merge sum#217L) AS sum(id#210L)#213L])
+- Exchange SinglePartition
 +- *(4) HashAggregate(keys=[], functions=[partial_sum(id#210L) AS sum#217L])
 +- *(4) Project [id#210L]
 +- *(4) SortMergeJoin [id#210L], [id#206L], Inner
 :- Sort [id#210L ASC NULLS FIRST], false, 0
 : +- Exchange hashpartitioning(id#210L, 200)
 : +- *(2) Project [(id#204L * 5) AS id#210L]
 : +- Exchange RoundRobinPartitioning(5)
 : +- *(1) Range (2, 10000000, step=2, splits=16)
 +- Sort [id#206L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(id#206L, 200)
 +- *(3) Range (2, 10000000, step=4, splits=16)

#### 스파크 잡 

<img src="https://i2.wp.com/techvidvan.com/tutorials/wp-content/uploads/sites/2/2019/11/Tasks-and-submitting-a-job-01.jpg?w=1200&ssl=1" width= 50% />
<br><br>

- 보통 액션 하나당 하나의 스파크 잡이 생성되면 액션은 항상 결과를 반환함
- <strong>스파크 잡은 일련의 스테이지</strong>로 나뉨
- 스테이지 수는 <strong>셔플 작업(넓은 의존성 트랜스포메이션)이 얼마나 많이 발생</strong>하는지에 따라 달라짐
  - 셔플은 데이터의 물리적 재분배 과정

#### 스테이지
- 스파크의 스테이지는 <strong>다수의 머신에서 동일한 연산을 수행하는 태스크의 그룹</strong>을 나타냄

- 가능한 한 많은 태스크(잡의 트랜스포메이션)를 동일한 스테이지로 묶으려 노력함

- 셔플 작업이 일어난 후에는 반드시 새로운 스테이지를 시작
  - 최종 결과를 계산하기 위해 스테이지 실행 순서를 계속 추적해야함
  
- <strong>위 잡의 스테이지 구성</strong>
  - Stage 23: 16/16
  - Stage 24: 16/16 
    - DataFrame 생성을 위해 사용한 range명령을 수행한 단계
    - range명령으로 df를 생성하면 기본적으로 16개의 파티션을 생성함
  
  - Stage 25: 5/5  
    - 파티션 재분배 단계(df1의 파티션 16 -> 5개)
      - df2(step12)은 이후에 트랜스포메이션이 없어서 생략된건가?
    - 데이터 셔플링으로 파티션 수를 변경함
    - DataFrame은 스테이지의 태스크 수와 같은 수의 파티션으로 재분배됨
    
  - Stage 26: 200/200  
  - Stage 27: 1/1
    - 조인(셔플) 수행 단계
    - 스파크 SQL은 spark.sql.suffle.partition 속성의 기본값이 200임
    - 그래서 스파크 잡이 실행되는 도중에 셔플을 수행하면 200개의 셔플 파티션이 기본적으로 생성
  
- 경험적으로 보았을 때 클러스터의 익스큐터 수보다 파티션 수를 더 크게 지정하는 것이 좋음
  - 대신 로컬 머신에서는 병렬로 처리할 수 있는 태스크 수가 제한적이므로 파티션 수를 작게 설정해야함

#### 태스크
- 스파크의 스테이지는 태스크로 구성됨 

- 각 태스크는 단일 익스큐터에서 실행할 데이터의 블록과 다수의 트랜스포메이션 조합으로 볼 수 있음
   - 만약 데이터셋이 거대한 하나의 파티션이면 하나의 태스크만 생성
   
- 즉, <strong>태스크는 데이터 단위(파티션)에 적용되는 연산 단위</strong>를 의미
  - 파티션 수를 늘리면 더 높은 병렬성을 얻을 수 있음
  - 최적화를 위한 가장 간단한 방법임(하지만 예외는 있음)

### 세부 실행 과정

#### 파이프라이닝

#### 셔플 결과 저장